# Testing homemade layers

This notebook contains tests for all the homemade layers. The tests takes some samples from the mni-
st dataset as a minibatch and compares the results from running this minibatch through the test CNN
to running the same minibatch through the homemade layers.

In [2]:
#importing relevant libraries

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
import os

%cd ..

from CNN_small_architecture import CNNSmall
from Tests.helper_functions import tokenize

/home/emil/Documents/bachelor/bachelorprojekt/Python


In [3]:
MNIST_test = datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())
test_set = [[data[0], tokenize(data[1])] for data in MNIST_test if data[1] in [1,2]][:1000]

batch_size = 1
test_loader = DataLoader(test_set, batch_size=batch_size)

In [4]:
model_original = CNNSmall()
model_original.eval()

path = "CNN_small"
load = True

if load and os.path.isfile(path):
    model_original.load_state_dict(torch.load(path))

preds = []

In [7]:
model_original.conv2.weight

Parameter containing:
tensor([[[[-4.4859e-02,  3.9975e-02,  5.8035e-02, -6.3092e-02, -1.0189e-01],
          [-2.5916e-02,  9.9018e-03,  4.3518e-02,  6.2601e-02, -5.4209e-02],
          [-6.5588e-02,  7.4557e-02,  1.4062e-01, -7.7475e-02,  1.2925e-01],
          [ 7.9538e-03,  4.8304e-02,  4.8660e-02, -8.2288e-02, -2.3752e-02],
          [-6.9754e-05, -3.3026e-02,  1.3297e-01,  2.7708e-03, -1.9535e-02]],

         [[-5.3181e-02, -9.3134e-02, -6.7886e-02,  6.0342e-02, -8.8757e-02],
          [-1.1812e-01, -3.2548e-02, -8.2704e-02,  1.0737e-01,  7.4177e-02],
          [-3.0274e-02, -5.2017e-02,  6.5842e-02,  2.8463e-02,  2.6924e-02],
          [ 6.9070e-02,  1.2792e-01, -3.2999e-02,  7.0541e-03, -4.1690e-02],
          [ 9.0487e-02,  1.0831e-02,  1.9193e-02, -3.9436e-02, -7.5359e-02]],

         [[-1.1584e-02,  3.2560e-02, -2.2781e-03,  1.5844e-02,  1.3116e-01],
          [-7.3840e-02, -5.5796e-02, -1.1778e-02, -6.3142e-03,  1.1828e-02],
          [-7.4526e-03,  3.4776e-02, -1.0390e-01, 

In [5]:
with torch.no_grad():
    for data in test_loader:
        image, label = data
        output = model_original(image)
        if torch.argmax(output) == torch.argmax(label):
            preds.append(1)
        else:
            preds.append(0)
        

In [6]:
preds.count(1)/len(preds)

0.994